**REPLICATION CODE: MOURIFIÉ/HENRY/MEANGO**

Start date: 10/11/2025
Author: Jordi Torres Vallverdú

Goal: idea is to replicate the analysis done by Mourifié et al (2020) in another context with another data. The idea is to seek new applications of their methodology.

Notes on the context: I am using data on Belgium region of Flandes in the 1990s. I have a panel of students observed from the beginning of high-school up to two years into higher education. Given that I don't observe future labor outcomes, I will only consider potential outcomes in terms of success probability of having completed the first year of higher education (this is a proxy for completion)


$D=\{1,0\}$, where 1 is entered stem high school track and 0 if entered non-stem high-school track

$Y=\{1,0\}$, where 1 is graduated from stem degree and 0 is graduated from non-stem degree

Z, set of instruments


In [1]:
using StatFiles, DataFrames

data_path = "/Users/jorditorresvallverdu/Library/Mobile Documents/com~apple~CloudDocs/tse/year1/papers_final_memoire/data/clean/roymodel_database.dta"

export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/set_identification/"


main = DataFrame(load(data_path))

Row,ID,hs_course1,hs_stem1,postsec_type2,postsec_stem2,c_par_beliefpar_HS_E,c_par_beliefpar_postHS,c_par_childage_homechange,c_par_notsureifB,c_par_yearsinkindergarten,c_par_repeatelementaryeduc,c_par_specialeducelem,c_par_income,c_par_home,c_par_spokenlanguage_dutch_fa,c_par_dialect_fa,c_par_spokenlanguage_dutch_mo,c_par_dialect_mo,c_par_parentschoose,c_par_childchoose,c_par_schoolchoose,c_par_otherchoose,c_par_nochoose,c_bao_advies_school_p0,c_bao_advies_progr_p0,c_bao_advies_welkprogr_p0,c_bao_belief_lkr_endpoint_p0,c_bao_special_p0,c_bao_result_dutch_p0,c_bao_result_math_p0,c_bao_result_all_p0,c_iq_male,c_iq_birth_year,c_iq_birth_month,c_iq_WB1,c_iq_NB1,c_iq_NIQ,c_iq_VIQ,c_iq_SIQ,c_iq_IQ,c_iq_opl,c_sskills_CON1,c_sskills_CON2,c_sskills_AGREE1,c_sskills_AGREE2,c_sskills_EXTRA1,c_sskills_CON3,c_sskills_AGREE3,c_sskills_EXTRA2,c_sskills_EXTRA3,c_sskills_workparents,c_sskills_income,c_sskills_IQ_discrete,c_iq_income_old
,Int32?,Float32?,Float32?,Float32?,Float32?,Int32?,Int32?,Int8?,Int8?,Int8?,Int8?,Int8?,Int32?,Int32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Int8?,Int8?,Int32?,Int32?,Float32?,Int32?,Int32?,Int32?,Int8?,Int16?,Int8?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Float32?,Float32?,Float32?,Float32?
1,11135,6.0,1.0,2.0,0.0,6,4,missing,0,3,0,0,4,1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1,1,7,0.0,5,4,4,1,1978,8,-0.418461,0.609975,91.4869,120.331,109.205,110.171,2,5,5,4,3,4,5,2,4,3,1.0,2.0,5.0,3.0
2,11137,6.0,0.0,1.0,0.0,6,4,missing,0,3,0,0,5,1,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0,1,1,1,0.0,5,4,5,0,1978,10,-0.332319,0.560348,97.4658,107.186,100.036,102.72,1,5,5,5,5,5,5,5,5,5,1.0,4.0,4.0,4.0
3,11154,6.0,1.0,2.0,0.0,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,1,1,1,7,0.0,5,5,5,1,1978,5,0.57565,1.49024,106.991,115.378,104.474,111.563,2,5,5,5,5,5,5,5,5,5,missing,1.0,5.0,1.0
4,11161,6.0,1.0,1.0,0.0,6,4,missing,0,4,0,0,5,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,5,5,missing,0,1978,2,0.221887,1.49837,110.288,115.582,99.4507,111.189,3,5,5,5,5,5,5,5,5,5,1.0,4.0,5.0,4.0
5,11162,6.0,1.0,1.0,0.0,6,5,missing,0,3,0,0,4,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,5,5,5,0,1978,2,0.988978,0.905644,118.363,100.334,90.7207,103.559,1,5,5,5,5,5,5,5,5,4,1.0,2.0,4.0,3.0
6,11163,6.0,1.0,2.0,0.0,6,4,missing,0,3,0,0,5,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,6,6,missing,0,1978,6,0.959115,0.984085,120.458,118.775,126.786,125.161,3,5,5,5,5,5,5,5,5,2,1.0,4.0,6.0,4.0
7,11166,6.0,1.0,2.0,0.0,6,missing,missing,0,3,0,0,missing,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,5,6,missing,0,1978,11,0.158003,1.39593,95.3408,91.9245,89.5506,90.9654,3,5,5,5,5,5,5,5,5,5,missing,1.0,3.0,1.0
8,11171,6.0,0.0,1.0,0.0,6,3,missing,0,3,0,0,4,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,6,6,missing,0,1978,5,0.83656,1.99263,110.01,125.206,121.763,123.113,2,5,5,5,5,5,5,5,5,5,1.0,2.0,6.0,3.0
9,11172,6.0,1.0,2.0,1.0,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,1,1,1,7,0.0,5,5,5,1,1978,10,1.0285,0.811891,108.808,96.97,99.3044,101.327,3,5,5,5,5,5,5,5,5,4,missing,1.0,4.0,1.0


**1. Data manipulation**

First we create a subsample of observations on gender and socio-economic status, just like they do in their exercise. I will create bins based on this: 

- Gender
- Income categories: 6 brackets
- Minority status (proxied by family background)


In [2]:
using DataFramesMeta, Statistics 

groups = groupby(main, [:c_sskills_income, :c_iq_male])
subsamples = [DataFrame(g) for g in groups]


6-element Vector{DataFrame}:
 540×54 DataFrame
 Row │ ID      hs_course1  hs_stem1  postsec_type2  postsec_stem2  c_par_belie ⋯
     │ Int32?  Float32?    Float32?  Float32?       Float32?       Int32?      ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  11166         6.0       1.0            2.0            0.0              ⋯
   2 │  11246         6.0       0.0            1.0            0.0
   3 │  11264         6.0       1.0            2.0            0.0
   4 │  11272         6.0       1.0            2.0            0.0
   5 │  11311         6.0       1.0            2.0            0.0              ⋯
   6 │  11347         6.0       0.0            1.0            0.0
   7 │  11426         6.0       1.0            1.0            1.0             
   8 │  11457         6.0       0.0            2.0            0.0
  ⋮  │   ⋮         ⋮          ⋮            ⋮              ⋮                 ⋮  ⋱
 534 │  42717         6.0       0.0            2.0        

**2. Test for monothonicity of E(Y/Z=z)**
This boils down to testing whether the probability of Y=1 is increasing in Z. Usual tests include Hu, Shi (2019), Chetverikhov (2013). These tests, however, are done under much more complicated cases as in mine: Hu et al, for example, account for a setting with Y being continuous and Z also being continuous; in my case, I just need to test over Z=1,2,3...

Given that the idea is to use set identification techniques, I will provide some evidence that this holds or not for a given subset of the data. Then I will focus mostly on the el(z) measure. 

In [3]:
size_subsample=10
size_p=3

m= zeros(Float64, size_subsample, size_p, size_p)

10×3×3 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [4]:
##Estimate probabilities 
zvals = sort(unique(main.c_iq_opl))

function cell_probs(df)
    k= length(zvals)
    p_k= zeros(Float64, k)
    n_k= zeros(Int, k)
    for z in zvals
        num = sum((df.postsec_stem2 .== 1) .& (df.c_iq_opl .== z))
        n_k[z] = sum(df.c_iq_opl .== z)
        p_k[z] = num / n_k[z]
    end

    return p_k, n_k
end


probs= [cell_probs(i) for i in subsamples]

6-element Vector{Tuple{Vector{Float64}, Vector{Int64}}}:
 ([0.23076923076923078, 0.3615819209039548, 0.43410852713178294], [234, 177, 129])
 ([0.5675675675675675, 0.5897435897435898, 0.5826771653543307], [148, 117, 127])
 ([0.25333333333333335, 0.38095238095238093, 0.29069767441860467], [75, 126, 86])
 ([0.6140350877192983, 0.5633802816901409, 0.6545454545454545], [57, 71, 55])
 ([0.25, 0.32323232323232326, 0.35], [24, 99, 360])
 ([0.5454545454545454, 0.47540983606557374, 0.570281124497992], [11, 61, 249])

In [5]:
#Estimate test

function pav3(pk, nk)
    p1, p2, p3 = pk
    n1, n2, n3 = nk

  #monothonicity holds
  if p1 ≤ p2 ≤ p3
    return pk
  end


  #case 1
  if p1>=p2
    
    θ_12= (p1*n1 + p2*n2)/(n1+n2)

    if p3>=θ_12
      return[θ_12, θ_12, p3]
    else 
    θ_123= (p1*n1 + p2*n2 + p3*n3)/(n1+n2+n3)

    return[θ_123,θ_123,θ_123]
    end 
  end 
  
  #case 2
  if p2>=p3

    θ_23= (p2*n2 + p3*n3)/(n2+n3)

    if p1<θ_23
      return[p1, θ_23, θ_23]
    else 
      θ_123= (p1*n1 + p2*n2 + p3*n3)/(n1+n2+n3)
     return[θ_123,θ_123,θ_123]
    end 

  end 


end


pav3 (generic function with 1 method)

In [6]:
#To check it works: 

pk = [0.40, 0.20, 0.50]
nk = [50, 60, 40]

pav3(pk, nk)


3-element Vector{Float64}:
 0.2909090909090909
 0.2909090909090909
 0.5

In [7]:
p_hat= zeros(length(subsamples), length(zvals))

for i in 1:length(subsamples)

    pk, nk = probs[i]

    p_hat[i,:]= pav3(pk,nk)
end 

p_hat

6×3 Matrix{Float64}:
 0.230769  0.361582  0.434109
 0.567568  0.586066  0.586066
 0.253333  0.34434   0.34434
 0.585938  0.585938  0.654545
 0.25      0.323232  0.35
 0.486111  0.486111  0.570281

In [8]:
n=zeros(length(subsamples))
T=zeros(length(subsamples))
[n[i]= nrow(subsamples[i]) for i in 1:length(subsamples)]

for i in 1:length(subsamples)

    pk, nk= probs[i]
    p_hat[i,:]= pav3(pk,nk)

    T[i]= sqrt(n[i])* maximum(pk .- p_hat[i,:])
end

T

6-element Vector{Float64}:
 0.0
 0.07282100100491927
 0.6202594605601705
 0.3800971214344959
 0.0
 1.0632250312587608

In [9]:
for i in 1:length(subsamples)
    println((i, unique(subsamples[i].c_sskills_income), unique(subsamples[i].c_iq_male), T[i]))
end


(1, Union{Missing, Float32}[1.0], Union{Missing, Int8}[0], 0.0)
(2, Union{Missing, Float32}[1.0], Union{Missing, Int8}[1], 0.07282100100491927)
(3, Union{Missing, Float32}[2.0], Union{Missing, Int8}[0], 0.6202594605601705)
(4, Union{Missing, Float32}[2.0], Union{Missing, Int8}[1], 0.3800971214344959)
(5, Union{Missing, Float32}[4.0], Union{Missing, Int8}[0], 0.0)
(6, Union{Missing, Float32}[4.0], Union{Missing, Int8}[1], 1.0632250312587608)


In [ ]:
using Distributions
using Random
Random.seed!(12345)   #seed


b_iter= 1000 # bootstrap

T_dist= zeros(length(subsamples), b_iter)

##set seed!!!

for b in 1:b_iter
  for i in 1:length(subsamples)

        dt = subsamples[i]   ##then we loop over i 
        p_iso = p_hat[i,:]   
        Z = dt.c_iq_opl  
        p_obs = p_iso[Z]   
        Y_b = rand.(Bernoulli.(p_obs))

        dt_boot = deepcopy(dt)       
        dt_boot.postsec_stem2 = Y_b   #simul replace

        pk_b, nk_b= cell_probs(dt_boot)
        p_iso_b= pav3(pk_b,nk_b)
        T_dist[i,b]= sqrt(n[i])* maximum(pk_b .- p_iso_b)
  end 
  
end

T_dist





6×1000 Matrix{Float64}:
 0.0       0.0       0.0        0.0       …  0.0       0.0       0.0
 0.0       0.425136  0.547891   0.402696     0.807259  0.283655  0.0
 0.0       0.0       0.104011   0.0          0.0       0.100206  0.0646896
 0.114956  0.412408  0.295689   0.247994     0.0       0.223799  0.234384
 0.0       0.535394  1.45175    0.357354     0.0       0.915719  0.0
 0.0       0.0       0.0678654  2.19199   …  0.0       3.43851   0.218863

In [11]:
using Statistics

c_value= zeros(length(subsamples))
[c_value[i]= quantile(T_dist[:, i], 0.95) for i in 1:length(subsamples)]


6-element Vector{Float64}:
 0.08621715193514179
 0.5078293056597845
 1.2257853073495553
 1.7446663993241094
 0.33932713763577377
 0.4057883817596935

In [12]:
t_result= zeros(length(subsamples))

for i in 1:length(subsamples)
    if (T[i]>=c_value[i])
        t_result[i]=1
    else 
        t_result[i]=0
    end
end 

t_result

6-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0